**1 处理前的准备**

1.1 导入必要的库

In [1]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
from collections import Counter
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MultiLabelBinarizer

1.2 确保输出目录存在

In [6]:
base_output_dir = Path("../data/advanced_analysis_results")
try:
    os.makedirs(base_output_dir, exist_ok=True)
    print(f"创建或确认 {base_output_dir} 目录成功")
except Exception as e:
    print(f"创建 {base_output_dir} 目录失败: {e}")
    exit()

# 加载 JSON 文件
def load_json(file_name):
    file_path = base_output_dir / file_name
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except Exception as e:
        print(f"加载 {file_path} 失败: {e}")
        return None

movies_data = load_json("summary_movies_results.json")
books_data = load_json("summary_books_results.json")
games_data = load_json("summary_games_results.json")

if not all([movies_data, books_data, games_data]):
    print("数据提示：无法加载所有 JSON 文件")
    exit()

创建或确认 ..\data\advanced_analysis_results 目录成功
加载 ..\data\advanced_analysis_results\summary_movies_results.json 失败: [Errno 2] No such file or directory: '..\\data\\advanced_analysis_results\\summary_movies_results.json'
加载 ..\data\advanced_analysis_results\summary_books_results.json 失败: [Errno 2] No such file or directory: '..\\data\\advanced_analysis_results\\summary_books_results.json'
加载 ..\data\advanced_analysis_results\summary_games_results.json 失败: [Errno 2] No such file or directory: '..\\data\\advanced_analysis_results\\summary_games_results.json'
数据提示：无法加载所有 JSON 文件


1.3 读取保存的 cleaned_data 字典

In [4]:
input_path = Path('../data/interim/cleaned_data.pkl')

if input_path.exists():
    with open(input_path, 'rb') as f:
        cleaned_data = pickle.load(f)
    print("✅ 成功加载 cleaned_data")
else:
    raise FileNotFoundError(f"未找到 cleaned_data.pkl，请先运行 02-cleaning.ipynb 并保存数据")

✅ 成功加载 cleaned_data


1.4 查看清洗后的数据

In [5]:
for sheet_name, df in cleaned_data.items():
    print(f"\n【{sheet_name}】前5行数据：")
    display(df.head())


【在玩】前5行数据：


,标题,简介,豆瓣评分,创建时间,我的评分,类型,发售时间
0,巫师3：狂猎 The Witcher 3: Wild Hunt,The Witcher III: Wild Hunt/猎魔人3：狂猎/角色扮演/冒险/动作/...,9.5,2025-05-06 16:11:28,5,"[角色扮演, 冒险, 动作]",2015-05-19
1,荒野大镖客：救赎2 Red Dead Redemption 2,碧血狂杀2/碧血狂殺2/血色救赎2/血色救贖2/Red Dead Redemption II...,9.7,2025-05-06 16:10:36,5,"[射击, 冒险, 动作]",2018-10-26



【想玩】前5行数据：


,标题,简介,豆瓣评分,创建时间,类型,发售时间
0,博德之门3 Baldur's Gate 3,角色扮演/冒险/策略/PC/Mac/PlayStation 5/Xbox Series X/...,9.6,2025-05-01 19:39:15,"[角色扮演, 冒险, 策略]",2023-08-03
1,苏丹的游戏,Sultan's Game/スルタンのゲーム/卡牌/模拟/角色扮演/策略/PC/2025-0...,暂无评分,2025-04-25 21:57:26,"[模拟, 角色扮演, 策略]",2025-03-31
2,光与影：33号远征队 Clair Obscur: Expedition 33,光與影：33號遠征隊/클레르 옵스퀴르: 33 원정대/角色扮演/冒险/动作/PC/Play...,暂无评分,2025-04-25 21:56:54,"[角色扮演, 冒险, 动作]",2025-04-24



【想看】前5行数据：


...

,标题,简介,豆瓣评分,创建时间,我的评分,类型,发售时间
0,山海旅人 The Rewinder,横版过关/角色扮演/冒险/PC/Mac/Xbox Series X/Nintendo Swi...,8.7,2025-05-06 16:14:28,4,"[横版过关, 角色扮演, 冒险]",2021-09-10
1,完蛋！我被美女包围了！,Love Is All Around/文字冒险/PC/Mac/iPhone/Android/...,7.7,2025-05-06 16:14:09,5,[文字冒险],2023-10-18
2,艾尔登法环 Elden Ring,艾爾登法環/老头环/上古之环/エルデンリング/角色扮演/冒险/动作/PC/PlayStati...,9.4,2025-05-06 16:13:05,5,"[角色扮演, 冒险, 动作]",2022-02-25
3,求生之路2 Left 4 Dead 2,L4D2/惡靈勢力 2/第一人称射击/射击/冒险/动作/PC/Mac/Linux/Xbox ...,9.0,2025-05-06 16:09:56,4,"[射击, 冒险, 动作]",2009-11-17
4,森林 The Forest,角色扮演/冒险/PC/PlayStation 4/Steam VR/Endnight Gam...,8.5,2025-05-06 16:09:12,4,"[角色扮演, 冒险]",2018-05-01



【看过】前5行数据：


,标题,简介,豆瓣评分,创建时间,我的评分,年份,国家/地区,类型,导演,主演
1,惊天魔盗团,2013 / 美国 法国 / 剧情 悬疑 犯罪 / 路易斯·莱特里尔 / 杰西·艾森伯格 艾...,7.8,2025-05-03 12:54:07,4,2013,"[美国, 法国]","[剧情, 悬疑, 犯罪]",[路易斯·莱特里尔],"[杰西·艾森伯格, 艾拉·菲舍尔]"
2,唐探1900,2025 / 中国大陆 中国香港 / 喜剧 动作 悬疑 / 陈思诚 戴墨 / 王宝强 刘昊然,6.5,2025-05-02 14:00:17,4,2025,"[中国大陆, 中国香港]","[喜剧, 动作, 悬疑]","[陈思诚, 戴墨]","[王宝强, 刘昊然]"
3,恶缘,2025 / 韩国 / 剧情 惊悚 犯罪 / 李日炯 / 朴海秀 申敏儿,8.4,2025-05-02 11:58:51,4,2025,[韩国],"[剧情, 惊悚, 犯罪]",[李日炯],"[朴海秀, 申敏儿]"
4,穿越火线,2020 / 中国大陆 / 剧情 奇幻 / 许宏宇 / 鹿晗 吴磊,7.9,2025-05-01 23:29:51,5,2020,[中国大陆],"[剧情, 奇幻]",[许宏宇],"[鹿晗, 吴磊]"
5,好小子们,2019 / 美国 / 喜剧 儿童 冒险 / 吉恩·斯图普尼兹基 / 雅各布·特伦布莱 基思...,7.2,2025-05-01 23:29:15,5,2019,[美国],"[喜剧, 儿童, 冒险]",[吉恩·斯图普尼兹基],"[雅各布·特伦布莱, 基思·L·威廉姆斯]"



【读过】前5行数据：


,标题,简介,豆瓣评分,创建时间,我的评分,作者,出版年份,出版社
0,13 67,陳浩基 / 2014 / 皇冠文化出版有限公司,9.1,2025-05-06 20:52:13,5,[陳浩基],2014,皇冠文化出版有限公司
1,无人生还,[英] 阿加莎・克里斯蒂 / 2008 / 人民文学出版社,9.0,2025-05-06 20:46:27,5,[阿加莎・克里斯蒂],2008,人民文学出版社
2,城南旧事,林海音 文 关维兴 图 / 2003 / 中国青年出版社,9.1,2025-05-06 20:46:05,5,"[林海音, 关维兴]",2003,中国青年出版社
3,东方快车谋杀案,[英] 阿加莎·克里斯蒂 / 2006 / 人民文学出版社,9.1,2025-05-06 20:45:53,5,[阿加莎·克里斯蒂],2006,人民文学出版社
4,麦田里的守望者,[美国] J. D. 塞林格 / 2007 / 译林出版社,8.1,2025-05-05 13:46:23,5,[塞林格],2007,译林出版社
